# RACNN GPU Demo

In [11]:
import numpy as np
import cv2
import sys
import os
import glob
import pickle
import time
import curacnn
import racnn_utils

In [12]:
# Parameters
model_path = './'

In [13]:
# COCO Class names
class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [14]:
def load_weights(model, weights, weight_num):
    '''
    Loading weights into the model
    '''
    for k in range(weight_num):
            w = weights[k]
            if w.ndim==1:
                w = np.reshape(w, (1,w.shape[0]))
            elif w.ndim==0:
                w = np.reshape(np.array([w]), (1,1))

            retval = model.load_weight(w, k)
            if (not retval):                
                break

    return retval    

In [19]:
def racnn_gpu_test(model_type, model_path, image_num):
    
    classes = len(class_names)
    model_type_index = {'vgg16':0, 'resnet50':1, 'vgg16_racnn':2, 'resnet50_racnn':3}
    
    if model_type=='vgg16':
        model_final = 'vgg16_coco_120.p'
        base_size = 224
        model_path = os.path.join(model_path,model_final)
        racnn_active = False       
        model_main_name = 'vgg'
    elif model_type=='resnet50':
        model_final = 'resnet50_coco_120.p'
        base_size = 256
        model_path = os.path.join(model_path,model_final)
        racnn_active = False
        model_main_name = 'resnet'
    elif model_type=='vgg16_racnn':
        model_final = 'vgg16_racnn_coco_120.p'
        base_size = 224
        model_path = os.path.join(model_path,model_final)
        racnn_active = True
        model_main_name = 'vgg'
    elif model_type=='resnet50_racnn':
        model_final = 'resnet50_racnn_coco_120.p'
        base_size = 256
        model_path = os.path.join(model_path,model_final)
        racnn_active = True
        model_main_name = 'resnet'
        
    model = curacnn.model()
    if not model.open(model_type_index[model_type]):
        print("cannot open curacnn")
        return
    
    weight_num = model.get_weight_num()
    # adding copy with block for fair speed comparison 
    model.set_speed_test(1)
    weights = pickle.load(open(model_path,"rb"))
    racnn_utils.rearrange_weights(weights, model_main_name, racnn_active, 1)
    if not load_weights(model, weights, weight_num):
        print("error in loading weights")
        return
    y_per = np.zeros((classes,), np.float32)
        
    img = cv2.imread("../test_data/%d/i_%d_%02d.png"%(base_size,base_size, image_num))
    if img is None:
        print('image not found')
        return
    
    x_test = np.float32(img)/255.0-0.5
    
    #warm up
    retval = model.predict(x_test, y_per)

    start_time = time.time()
    retval = model.predict(x_test, y_per)
    ellaped = time.time()-start_time
    if (not retval):
        print("error in prediction")
        return

    print("class: %s, prob: %2.2f ,elapsed time: %2.2f sec"%(class_names[int(np.argmax(y_per))], np.max(y_per), ellaped))
    

### VGG16 Model

In [20]:
racnn_gpu_test('vgg16', model_path, 0)

class: bowl, prob: 1.00 ,elapsed time: 0.04 sec


# Resnet Model

In [21]:
racnn_gpu_test('resnet50', model_path, 0)

class: bowl, prob: 1.00 ,elapsed time: 0.02 sec


# VGG16-RACNN Model

In [22]:
racnn_gpu_test('vgg16_racnn', model_path, 0)

class: bowl, prob: 1.00 ,elapsed time: 0.04 sec


# Resnet50-RACNN Model

In [23]:
racnn_gpu_test('resnet50_racnn', model_path, 0)

class: bowl, prob: 1.00 ,elapsed time: 0.02 sec
